In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
import re
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
import os
import json
import pandas as pd
import requests

# Ollama 관련 설정 및 함수
try:
    import ollama
    OLLAMA_AVAILABLE = True
    # Ollama 서버 설정
    ollama_host = "http://sg016:11434"
    ollama_client = ollama.Client(host=ollama_host)
except ImportError:
    OLLAMA_AVAILABLE = False
    print("Ollama 패키지가 설치되어 있지 않습니다. pip install ollama로 설치하세요.")

def check_ollama_server():
    """Ollama 서버 연결 상태 확인"""
    if not OLLAMA_AVAILABLE:
        return False
        
    try:
        response = requests.get(f"{ollama_host}/api/version")
        print(f"Ollama 서버 연결됨: 버전 {response.json().get('version', '알 수 없음')}")
        return True
    except Exception as e:
        print(f"Ollama 서버 연결 실패: {e}")
        return False

# Hugging Face 모델 관련 함수
def load_model_and_tokenizer(model_path, use_local=True, dtype=torch.bfloat16):
    """모델과 토크나이저를 로드하는 함수"""
    print(f"모델 로딩 중: {model_path}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            model_path, 
            local_files_only=use_local
        )
        
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            local_files_only=use_local,
            torch_dtype=dtype,
            device_map="auto"
        )
        print("모델 로딩 완료!")
        return model, tokenizer
    except Exception as e:
        print(f"모델 로딩 중 오류 발생: {e}")
        return None, None

def predict_similarity_with_model(model, tokenizer, sentence1, sentence2):
    """특정 모델을 사용하여 두 문장의 유사도를 예측하는 함수"""
    prompt = f"Analyze the following sentence pair and provide a similarity score between 0 and 5, where 0 means no similarity and 5 means identical.\n\nSentence 1: {sentence1}\nSentence 2: {sentence2}\n\nSimilarity Score:"
    
    # 토크나이즈
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 추론 수행
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=15,
            temperature=0.1,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 결과 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 프롬프트 이후의 생성된 텍스트만 추출
    response = response[len(prompt):]
    
    # 숫자 추출 시도
    try:
        # 0~5 사이의 숫자 찾기
        match = re.search(r'([0-5](\.\d+)?)', response)
        if match:
            predicted_score = float(match.group(1))
        else:
            # 숫자를 찾지 못한 경우 기본값 설정
            print(f"점수 추출 불가: '{response}'. 기본값 사용.")
            predicted_score = 2.5  # 기본값
    except Exception as e:
        print(f"점수 추출 오류: {e}. 응답: '{response}'")
        predicted_score = 2.5  # 오류 발생 시 기본값
    
    # 0~5 범위로 제한
    predicted_score = max(0, min(5, predicted_score))
    
    return predicted_score

def predict_similarity_ollama(sentence1, sentence2, model_name="llama3.2", language="en"):
    """Ollama API를 사용하여 두 문장의 유사도를 예측하는 함수"""
    if not OLLAMA_AVAILABLE:
        print("Ollama 패키지를 사용할 수 없습니다.")
        return 2.5
 
    prompt = f"Analyze the following sentence pair and provide a similarity score between 0 and 5, where 0 means no similarity and 5 means identical.\n\nSentence 1: {sentence1}\nSentence 2: {sentence2}\n\nReply with only a number."
    
    try:
        # 문장 쌍을 프롬프트와 함께 모델에 입력
        response = ollama_client.chat(
            model=model_name, 
            messages=[{
                'role': 'user',
                'content': prompt,
            }],
            options={
                'temperature': 0.1,  # 낮은 온도로 일관된 결과
                'top_p': 0.95,
            }
        )
        
        # 응답에서 점수 추출
        content = response['message']['content'].strip()
        
        # 정규 표현식으로 숫자 추출
        match = re.search(r'([0-5](\.\d+)?)', content)
        if match:
            predicted_score = float(match.group(1))
        else:
            # 숫자를 찾지 못한 경우 2.5로 기본 설정
            print(f"응답에서 점수를 추출할 수 없습니다: '{content}'. 기본값 사용.")
            predicted_score = 2.5
            
        # 0~5 범위로 제한
        predicted_score = max(0, min(5, predicted_score))
        
        return predicted_score
        
    except Exception as e:
        print(f"예측 중 오류 발생: {e}")
        return 2.5  # 오류 발생 시 기본값

# 통합 평가 함수
def evaluate_model_on_klue_sts(model_path, eval_samples=100, save_results=True, output_dir="results"):
    """KLUE STS 데이터셋을 사용하여 모델(HF 또는 Ollama)을 평가하는 함수"""
    # 모델 정보 추출
    model_type = model_info.get("type", "hf")  # 기본값은 huggingface
    model_name = model_info.get("name", "Unknown")
    language = model_info.get("language", "en")
    
    if model_type == "ollama":
        # Ollama 서버 확인
        if not check_ollama_server():
            print("Ollama 서버에 연결할 수 없어 평가를 건너뜁니다.")
            return None
        model_path = model_info.get("path", model_name)  # Ollama의 경우 path는 모델명과 동일
    else:  # huggingface
        model_path = model_info.get("path")
        use_local = model_info.get("local", True)
        
        # 모델 및 토크나이저 로드 (HF 모델만)
        model, tokenizer = load_model_and_tokenizer(model_path, use_local)
        if model is None or tokenizer is None:
            return None
        
        # 평가 모드로 전환
        model.eval()
    
    # 데이터셋 로드
    print("KLUE STS 데이터셋 로드 중...")
    dataset = load_dataset('klue', 'sts')
    
    # 예측 및 레이블 저장용 리스트
    predictions = []
    labels = []
    examples_data = []  # 디버깅 및 분석용 데이터 저장
    
    # 시작 시간 기록
    start_time = time.time()
    
    # 평가할 샘플 수 설정
    if eval_samples is None or eval_samples > len(dataset['validation']):
        eval_samples = len(dataset['validation'])
    
    print(f"총 {eval_samples}개 샘플 평가 시작...")
    for i, example in enumerate(dataset['validation']):
        if i >= eval_samples:
            break
            
        sentence1 = example['sentence1']
        sentence2 = example['sentence2']
        label = example['labels']['label']  # 실제 유사도 점수
        
        # 두 문장에 대해 유사도 예측 (모델 타입에 따라 다른 함수 사용)
        if model_type == "ollama":
            pred = predict_similarity_ollama(sentence1, sentence2, model_info["path"])
        else:  # huggingface
            pred = predict_similarity_with_model(model, tokenizer, sentence1, sentence2)
            
        predictions.append(pred)
        labels.append(label)
        
        # 예제 데이터 저장
        examples_data.append({
            'sentence1': sentence1,
            'sentence2': sentence2,
            'true_score': label,
            'predicted_score': pred
        })
        
        # 진행 상황 출력
        if (i+1) % 10 == 0:
            elapsed = time.time() - start_time
            avg_time = elapsed / (i+1)
            remaining = avg_time * (eval_samples - (i+1))
            print(f"처리됨: {i+1}/{eval_samples} 예제 | 평균 시간: {avg_time:.2f}초/예제 | 남은 시간: {remaining/60:.2f}분")
    
    # 평가 지표 계산
    mse = mean_squared_error(labels, predictions)
    mae = mean_absolute_error(labels, predictions)
    
    # 결과 출력
    print("\n평가 결과:")
    print(f"모델: {model_name}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    
    # 결과 저장
    if save_results:
        # 결과 디렉토리 생성
        os.makedirs(output_dir, exist_ok=True)
        
        # 파일명에 사용할 안전한 모델명 생성
        safe_model_name = model_name.replace("/", "-")
        
        # 결과 요약 저장
        results = {
            'model_name': model_name,
            'model_path': model_path,
            'model_type': model_type,
            'eval_samples': eval_samples,
            'mse': float(mse),
            'mae': float(mae),
            'timestamp': time.strftime("%Y-%m-%d %H:%M:%S")
        }
        
        with open(f"{output_dir}/{safe_model_name}_summary.json", 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        # 세부적인 예제별 결과 저장
        df = pd.DataFrame(examples_data)
        df.to_csv(f"{output_dir}/{safe_model_name}_detailed.csv", index=False, encoding='utf-8')
        
        print(f"결과가 {output_dir} 디렉토리에 저장되었습니다.")
    
    return {"mse": mse, "mae": mae}

In [ ]:
# 평가할 모델 경로 목록
models_to_evaluate = [
    {
        "type": "hf",
        "path": "allenai/OLMo-7B-hf",
        "name": "OLMo-7B",
    },
    {
        "type": "hf",
        "path": "fine-tuned-models/fine-tuned-olmo7B-v12-80000",
        "name": "OLMo-7B-Fine-Tuned",
    },
    {
        "type": "hf",
        "path": "allenai/OLMo-1B-hf",
        "name": "OLMo-1B",
    },
    {
        "type": "hf",
        "path": "fine-tuned-models/fine-tuned-olmo1B-80000-v11",
        "name": "OLMo-1B-Fine-Tuned",
    },
    {
        "type": "ollama",
        "path": "llama3.2",
        "name": "Llama-3.2",
    }
]

# 결과 저장을 위한 디렉토리
results_dir = "klue_sts_results"

# 각 모델 평가
all_results = {}
for model_info in models_to_evaluate:
    print(f"\n===== {model_info['name']} 평가 시작 =====")
    results = evaluate_model_on_klue_sts(
        model_path=model_info["path"],
        eval_samples=len(dataset['validation']),  # 전체 검증 세트 사용
        save_results=True,
        output_dir=results_dir
    )
    if results:
        all_results[model_info["name"]] = results

# 모든 모델 결과 비교
if len(all_results) > 1:
    print("\n===== 모델 성능 비교 =====")
    for model_name, metrics in all_results.items():
        print(f"{model_name}: MSE={metrics['mse']:.4f}, MAE={metrics['mae']:.4f}")



===== OLMo-7B 평가 시작 =====
모델 로딩 중: allenai/OLMo-7B-hf


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

모델 로딩 완료!
KLUE STS 데이터셋 로드 중...
총 519개 샘플 평가 시작...
처리됨: 10/519 예제 | 평균 시간: 0.50초/예제 | 남은 시간: 4.28분
처리됨: 20/519 예제 | 평균 시간: 0.49초/예제 | 남은 시간: 4.07분
처리됨: 30/519 예제 | 평균 시간: 0.49초/예제 | 남은 시간: 3.96분
처리됨: 40/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.87분
처리됨: 50/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.78분
처리됨: 60/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.70분
처리됨: 70/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.61분
처리됨: 80/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.53분
처리됨: 90/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.43분
처리됨: 100/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.35분
처리됨: 110/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.27분
처리됨: 120/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.19분
처리됨: 130/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.11분
처리됨: 140/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.02분
처리됨: 150/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 2.94분
처리됨: 160/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 2.86분
처리됨: 170/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 2.79분
처리됨: 180/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 2.72분
처리됨: 190/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 2.64분
처리됨: 200/519 예제 | 평균 시간: 0.

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

모델 로딩 완료!
KLUE STS 데이터셋 로드 중...
총 519개 샘플 평가 시작...
점수 추출 불가: ' The image depicts a man and woman sitting at a table, with the'. 기본값 사용.
점수 추출 불가: ' The image depicts a man and woman sitting at a table, with the'. 기본값 사용.
점수 추출 불가: ' The image shows a man with glasses and a suit jacket, possibly in'. 기본값 사용.
점수 추출 불가: '

The image appears to be a still from a television show or movie'. 기본값 사용.
점수 추출 불가: ' The image depicts a man and woman standing in a room, with the'. 기본값 사용.
점수 추출 불가: ' The image depicts a man sitting at a table with a glass of wine'. 기본값 사용.
점수 추출 불가: ' The image shows a man with dark hair wearing a black shirt, possibly'. 기본값 사용.
점수 추출 불가: ' The image shows a man and woman sitting at a table with a laptop'. 기본값 사용.
점수 추출 불가: ' This image features a young man with dark hair, wearing a black jacket'. 기본값 사용.
점수 추출 불가: ' The image depicts a man and woman sitting at a table, with the'. 기본값 사용.
처리됨: 10/519 예제 | 평균 시간: 0.47초/예제 | 남은 시간: 4.03분
점수 추출 불가: ' The image shows a

In [15]:
#######################################################
# KLUE STS dataset
#######################################################
from datasets import load_dataset

klue_sts = load_dataset('klue', 'sts')
   
# 데이터셋 확인 (예: train 데이터셋의 50번째 샘플 확인)
print(klue_sts['train'])  # 또는 'validation' 또는 'test'에 접근 가능

# 예시로 'validation' 데이터셋을 확인
print(klue_sts['validation'])

for i in range(10):
    print(f"{i}th data")
    print("sentence1: ", klue_sts['train'][i]["sentence1"])
    print("sentence2: ", klue_sts['train'][i]["sentence2"])
    print("similarity: ", klue_sts['train'][i]["labels"]["label"])

Dataset({
    features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
    num_rows: 11668
})
Dataset({
    features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
    num_rows: 519
})
0th data
sentence1:  숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.
sentence2:  숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.
similarity:  3.7
1th data
sentence1:  위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.
sentence2:  시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.
similarity:  0.0
2th data
sentence1:  회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.
sentence2:  사람들이 주로 네이버 메일을 쓰는 이유를 알려줘
similarity:  0.3
3th data
sentence1:  긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업성공패키지, 청년구직활동지원금, 긴급복지지원제도 지원금과는 중복 수급이 불가능하다.
sentence2:  고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.
similarity:  0.6
4th data
sentence1:  호스트의 답장이 늦으나, 개선될 것으로 보입니다.
sentence2:  호스트 응답이 늦었지만 개선될 것으로 보입니다.
similarity:  4.7
5th data
sentence1:  정부가 새로운 일자리를 직접 창출하는 노력도 배가하겠습니다.
sentence2:  세계에서 우리만큼 오랜 역사와 문화를 공유하는 가까운 이웃이 없습니다.
similarity:  0.0
6th d

In [19]:
klue_dp = load_dataset('klue', 'dp')
# print(klue_dp)

print(klue_dp['validation'][0]['sentence'])
print(klue_dp['validation'][0]['index'])
print(klue_dp['validation'][0]['word_form'])
print(klue_dp['validation'][0]['lemma'])
print(klue_dp['validation'][0]['pos'])
print(klue_dp['validation'][0]['head'])
print(klue_dp['validation'][0]['deprel'])

'K팝스타3’ 유희열이 홍정희의 탈락에 눈물을 흘렸다.
[1, 2, 3, 4, 5, 6]
["'K팝스타3’", '유희열이', '홍정희의', '탈락에', '눈물을', '흘렸다.']
["' K 팝스타 3 ’", '유희열 이', '홍정희 의', '탈락 에', '눈물 을', '흘리 었 다 .']
['SS+SL+NNP+SN+SS', 'NNP+JKS', 'NNP+JKG', 'NNG+JKB', 'NNG+JKO', 'VV+EP+EF+SF']
[2, 6, 4, 6, 6, 0]
['NP', 'NP_SBJ', 'NP_MOD', 'NP_AJT', 'NP_OBJ', 'VP']


In [23]:
klue_mrc = load_dataset('klue', 'mrc')
# print(klue_mrc)

print(klue_mrc['validation'][0]['title'])
print(klue_mrc['validation'][0]['context'])
print(klue_mrc['validation'][0]['news_category'])
print(klue_mrc['validation'][0]['source'])
print(klue_mrc['validation'][0]['guid'])
print(klue_mrc['validation'][0]['is_impossible'])
print(klue_mrc['validation'][0]['question_type'])
print(klue_mrc['validation'][0]['question'])
print(klue_mrc['validation'][0]['answers'])

BMW 코리아, 창립 25주년 기념 ‘BMW 코리아 25주년 에디션’ 한정 출시
BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다.
자동차
acrofan
klue-mrc-v1_dev_01891


In [25]:
klue_ner = load_dataset('klue', 'ner')
# print(klue_ner)

print(klue_ner['validation'][0]['sentence'])
print(klue_ner['validation'][0]['tokens'])
print(klue_ner['validation'][0]['ner_tags'])

<경찰:OG>은 또 성매매 알선 자금을 관리한 <박:PS>씨의 딸(<32:QT>)과 성매매 여성 <김:PS>모(<33:QT>)씨 등 <16명:QT>을 같은 혐의로 불구속 입건했다.
['경', '찰', '은', ' ', '또', ' ', '성', '매', '매', ' ', '알', '선', ' ', '자', '금', '을', ' ', '관', '리', '한', ' ', '박', '씨', '의', ' ', '딸', '(', '3', '2', ')', '과', ' ', '성', '매', '매', ' ', '여', '성', ' ', '김', '모', '(', '3', '3', ')', '씨', ' ', '등', ' ', '1', '6', '명', '을', ' ', '같', '은', ' ', '혐', '의', '로', ' ', '불', '구', '속', ' ', '입', '건', '했', '다', '.']
[4, 5, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 6, 12, 12, 12, 12, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 6, 12, 12, 8, 9, 12, 12, 12, 12, 12, 8, 9, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]


In [27]:
klue_nli = load_dataset('klue', 'nli')
# print(klue_nli)

print(klue_nli['validation'][0]['guid'])
print(klue_nli['validation'][0]['source'])
print(klue_nli['validation'][0]['premise'])
print(klue_nli['validation'][0]['hypothesis'])
print(klue_nli['validation'][0]['label'])

klue-nli-v1_dev_00000
airbnb
흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.
어떤 방에서도 흡연은 금지됩니다.
2


In [31]:
klue_re = load_dataset('klue', 're')
# print(klue_re)

print(klue_re['validation'][0]['guid'])
print(klue_re['validation'][0]['sentence'])
print(klue_re['validation'][0]['subject_entity'])
print(klue_re['validation'][0]['object_entity'])
print(klue_re['validation'][0]['label'])
print(klue_re['validation'][0]['source'])

klue-re-v1_dev_00000
20대 남성 A(26)씨가 아버지 치료비를 위해 B(30)씨가 모아둔 돈을 훔쳐 인터넷 방송 BJ에게 '별풍선'으로 쏜 사실이 알려졌다.
{'word': 'A', 'start_idx': 7, 'end_idx': 7, 'type': 'PER'}
{'word': '30', 'start_idx': 29, 'end_idx': 30, 'type': 'NOH'}
0
wikitree


In [32]:
klue_wos = load_dataset('klue', 'wos')
# print(klue_wos)

print(klue_wos['validation'][0]['guid'])
print(klue_wos['validation'][0]['domains'])
print(klue_wos['validation'][0]['dialogue'])

wos-v1_dev_00000
['관광', '식당', '숙소']
[{'role': 'user', 'text': '명동 쇼핑 거리에 대해 물어볼게 있는데 영업시간이랑 입장료, 주소를 알려주세요.', 'state': ['관광-이름-명동 쇼핑 거리']}, {'role': 'sys', 'text': '먼저 명동 쇼핑 거리에 영업시간은 09시에서 20시까지입니다. 그리고 입장료는 10000원이고 주소는 서울 중구 38345입니다.', 'state': []}, {'role': 'user', 'text': '그리고 다음으로 서울 동쪽에 주류를 판매하는 일식당을 찾아주세요. 가격대는 상관없습니다.', 'state': ['관광-이름-명동 쇼핑 거리', '식당-가격대-dontcare', '식당-지역-서울 동쪽', '식당-종류-일식당', '식당-주류 판매-yes']}, {'role': 'sys', 'text': '그럼 올림픽공원역 근처에 있는 적당한 가격대의 매일식사를 추천합니다.', 'state': []}, {'role': 'user', 'text': '그럼 그 식당으로 금요일 23시20분에 두명 예약해주세요.', 'state': ['관광-이름-명동 쇼핑 거리', '식당-가격대-dontcare', '식당-지역-서울 동쪽', '식당-종류-일식당', '식당-주류 판매-yes', '식당-예약 요일-금요일', '식당-예약 시간-23:20', '식당-예약 명수-2', '식당-이름-매일식사']}, {'role': 'sys', 'text': "네 예약번호 'ZLUP1'로 예약되었습니다.", 'state': []}, {'role': 'user', 'text': '그럼 그 식당 평점하고 역에서 걷는 시간을 알려주세요.', 'state': ['관광-이름-명동 쇼핑 거리', '식당-가격대-dontcare', '식당-지역-서울 동쪽', '식당-종류-일식당', '식당-주류 판매-yes', '식당-예약 요일-금요일', '식당-예약 시간-23:20', '식당-예약 명수-2', '식당-이름-매일식사']},

In [33]:
klue_ynat = load_dataset('klue', 'ynat')
# print(klue_ynat)

print(klue_ynat['validation'][0]['guid'])
print(klue_ynat['validation'][0]['title'])
print(klue_ynat['validation'][0]['label'])
print(klue_ynat['validation'][0]['url'])
print(klue_ynat['validation'][0]['date'])

ynat-v1_dev_00000
5억원 무이자 융자는 되고 7천만원 이사비는 안된다
2
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=260&oid=001&aid=0009563542
2017.09.21. 오후 5:09
